In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime,timedelta
import pytz

In [ ]:
API_KEY='54adafa78659ef682f7075e7af64e1fb'
BASE_URL='https://api.openweathermap.org/data/2.5/'

Fetching Current DAta

In [ ]:
def get_current_weather(city):
  url=f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
  response=requests.get(url)
  data= response.json()
  return {
      'city': data['name'],
      'current_temp': round(data['main']['temp']),
      'feels_like': round(data['main']['feels_like']),
      'temp_min': round(data['main']['temp_min']),
      'temp_max': round(data['main']['temp_max']),
      'pressure': data['main']['pressure'],
      'humidity': round(data['main']['humidity']),
      'wind_speed': data['wind']['speed'],
      'description': data['weather'][0]['description'],
      'country': data['sys']['country'],
      'wind_gust_dir': data['wind']['deg'],
      'wind_gust_speed': data['wind']['speed'],
  }

reading historical data

In [ ]:
def read_historical_data(filename):
  df=pd.read_csv(filename)
  df=df.dropna()
  df=df.drop_duplicates()
  return df

testing and preparing data for ML

In [ ]:
def prepare_data(data):
  le=LabelEncoder()
  #data['city']=le.fit_transform(data['city'])
  data['WindGustDir']=le.fit_transform(data['WindGustDir'])
  data['RainTomorrow']=le.fit_transform(data['RainTomorrow'])

  X=data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed','Humidity','Pressure','Temp']]
  Y=data['RainTomorrow']
  return X,Y,le

train rain prediction model

In [ ]:
def train_rain_model(X,Y):
  X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
  model=RandomForestClassifier(n_estimators=100,random_state=42)
  model.fit(X_train,Y_train)
  Y_pred=model.predict(X_test)
  print("Mean Square error for rain model")
  print(mean_squared_error(Y_test,Y_pred))
  return model

prepare regression data

In [ ]:
def prepare_regression_data(data,feature):
  X,Y=[],[]
  for i in range(len(data)-1):
    X.append(data[feature].iloc[i])
    Y.append(data[feature].iloc[i+1])
  return np.array(X).reshape(-1,1),np.array(Y)

train regression data

In [ ]:
def train_regression_model(X,Y):
  model=RandomForestRegressor(n_estimators=100,random_state=42)
  model.fit(X,Y)
  return model

future prediction

In [ ]:
def predict_future(model,current_value):
  predictions=[current_value]
  for i in range(5):
    next_value=model.predict(np.array([[predictions[-1]]]))
    predictions.append(next_value[0])
  return predictions[1:]

Weather Analysis

In [ ]:
historical_data = read_historical_data('/content/weather.csv')
print(historical_data.columns)

Index(['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity',
       'Pressure', 'Temp', 'RainTomorrow'],
      dtype='object')


In [ ]:
def weather_view():
  city=input("Enter city name: ")
  current_weather=get_current_weather(city)

  #load historical data
  historical_data=read_historical_data('/content/weather.csv')
  #prepare
  X,Y,le=prepare_data(historical_data)
  #train
  rain_model=train_rain_model(X,Y)
  # Map wind direction to compass points
  wind_deg = current_weather['wind_gust_dir'] % 360
  compass_points = [
      ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
      ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
      ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
      ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
      ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
      ("NNW", 326.25, 348.75)
  ]

  compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)

  compass_direction_encoded=le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

  current_data={
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir':compass_direction_encoded,
      'WindGustSpeed': current_weather['wind_gust_speed'],
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp'],

  }

  current_df=pd.DataFrame([current_data])
  #rain predictor
  rain_prediction=rain_model.predict(current_df)[0]
  #prepare regression data
  X_temp,Y_temp=prepare_regression_data(historical_data,'Temp')
  X_hum,Y_hum=prepare_regression_data(historical_data,'Humidity')
  temp_model=train_regression_model(X_temp,Y_temp)
  hum_model=train_regression_model(X_hum,Y_hum)
  #predict future temp and humidity
  future_temp= predict_future(temp_model,current_weather['temp_min'])
  future_humidity= predict_future(hum_model,current_weather['humidity'])

  #prepare timezone

  timezone=pytz.timezone('Asia/Kolkata')
  now=datetime.now(timezone)
  next_hour= now+timedelta(hours=1)
  next_hour=next_hour.replace(minute=0,second=0,microsecond=0)
  future_times= [(next_hour+timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  #results

  print(f"City: {city}, {current_weather['country']}")
  print(f"Current Temperature: {current_weather['current_temp']}°C")
  print(f"Feels Like: {current_weather['feels_like']}°C")
  print(f"Minimum Temperature: {current_weather['temp_min']}°C")
  print(f"Maximum Temperature: {current_weather['temp_max']}°C")
  print(f"Pressure: {current_weather['pressure']} hPa")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather Description: {current_weather['description']}")
  print(f"Rain Prediction: {'Yes' if rain_prediction else 'No' }")

  print("\n Future Temprature Predictions: ")

  for time,temp in zip(future_times,future_temp):
    print(f"{time}: {round(temp,1)}°C")

  print("\n Future Humidity Predictions: ")

  for time,humidity in zip(future_times,future_humidity):
    print(f"{time}: {round(humidity, 1)}%")

weather_view()

Enter city name: Kolkata
Mean Square error for rain model
0.1506849315068493
City: Kolkata, IN
Current Temperature: 30°C
Feels Like: 37°C
Minimum Temperature: 30°C
Maximum Temperature: 30°C
Pressure: 1001 hPa
Humidity: 79%
Weather Description: haze
Rain Prediction: Yes

 Future Temprature Predictions: 
14:00: 28.5°C
15:00: 22.0°C
16:00: 23.9°C
17:00: 21.4°C
18:00: 22.7°C

 Future Humidity Predictions: 
14:00: 54.0%
15:00: 58.1%
16:00: 54.2%
17:00: 58.1%
18:00: 54.2%
